In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('kinetic_data.xlsx')

In [3]:
# properties
vt = 145 # mL, total reactor volume
rho_h2o = 1 # g/mL, water density
mw_h2o = 18 # g/mol, water molweight
rho_gly = 1.26 # g/mL, glycerol density
mw_gly = 92 # g/mol, glycerol molweight
mw_koh = 55 # g/mol, KOH molweight
mw_h2 = 2 # g/mol, hydrogen molweight

# h2 solubility parameter
p00 = 0.2638
p01 = 5.443e-4
p02 = -2.45e-7
p10 = -1.545e-3
p20 = 2.205e-6
p11 = 9.105e-7

In [4]:
# treatment
df['W_H2O [g]'] = df['VL_H2O [mL]']
df['Ni_H2O [mmol]'] = df['W_H2O [g]']/mw_h2o*1000
df['EG [%]'] = df['ME [%]']
df['TOT [%]'] = df['LA [%]'] + df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]']
df['Wi [g]'] = df['W_H2O [g]'] + df['Ni_Gly [mmol]']*mw_gly/1000 + df['Ni_KOH [mmol]']*mw_koh/1000
df['VL [mL]'] = df['VL_H2O [mL]'] +df['Ni_Gly [mmol]']*mw_gly/1000/rho_gly
df['VG [mL]'] = vt - df['VL [mL]']

# selectivity
df_sel = pd.DataFrame()
df_sel['LA [%]'] = df['LA [%]']
df_sel['PDO [%]'] = df['PDO [%]'] + (100-df['TOT [%]'])*df['PDO [%]']/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['FA [%]'] = df['FA [%]'] + (100-df['TOT [%]'])*(df['FA [%]']+df['GA [%]'])/2/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['GA [%]'] = df['GA [%]'] + (100-df['TOT [%]'])*(df['FA [%]']+df['GA [%]'])/2/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['ME [%]'] = df['ME [%]'] + (100-df['TOT [%]'])*df['ME [%]']/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['EG [%]'] = df_sel['ME [%]']
df_sel.iloc[0] = 0
df_sel['TOT [%]'] = df_sel['LA [%]'] + df_sel['PDO [%]'] + (df_sel['FA [%]'] + df_sel['GA [%]'])/2 + df_sel['ME [%]']

# total mole number
df_mol = pd.DataFrame()
df_mol['N_Gly [mmol]'] = df['Ni_Gly [mmol]']*(100-df['Conv [%]'])/100
df_mol['N_LA [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['LA [%]']/100
df_mol['N_PDO [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['PDO [%]']/100
df_mol['N_FA [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['FA [%]']/100
df_mol['N_GA [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['GA [%]']/100
df_mol['N_ME [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['ME [%]']/100
df_mol['N_EG [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['EG [%]']/100
df_mol['N_KOH [mmol]'] = df['Ni_KOH [mmol]'] - df_mol['N_LA [mmol]'] - df_mol['N_FA [mmol]'] - df_mol['N_GA [mmol]']
df_mol['N_H2O [mmol]'] = df['Ni_H2O [mmol]'] + df_mol['N_LA [mmol]'] + df_mol['N_PDO [mmol]']
df_mol['N_H2 [mmol]'] = df_mol['N_LA [mmol]'] + df_mol['N_FA [mmol]']*1.5 + df_mol['N_GA [mmol]']*1.5 - df_mol['N_ME [mmol]'] - df_mol['N_PDO [mmol]']
df_mol['N_TOT [mmol]'] = df_mol.sum(axis=1)

# hydrogen solubility

df_h2 = pd.DataFrame()
df_h2['P_H2O [bar]'] = 10**(8.14019-1810.94/(244.485+df['Temp [degC]']))/750.062 # Antoine equation
df_h2['NG_H2O [mmol]'] = 0.0
df_h2['P_H2 [bar]'] = 0.0
for i in range(len(df_h2)):
    vg = 8.314*(df['Temp [degC]'].iloc[i]+273.15)/(df_h2['P_H2O [bar]'].iloc[i]*1e5)
    df_h2.loc[i,'NG_H2O [mmol]'] = 1/(vg*1e3)*df.loc[i,'VG [mL]']
    if i != 0:
        df_h2.loc[i, 'P_H2 [bar]'] = (df_mol['N_H2 [mmol]'].iloc[i]/1e3)/(df.loc[i,'VG [mL]']/1e6)*8.314*(df['Temp [degC]'].iloc[i]+273.15)/1e5
    else:
        pass
df_h2['S_H2 [mol/kg]'] = p00 + (df['Temp [degC]']+273.15)*p10 + df_h2['P_H2 [bar]']*p01 + (df['Temp [degC]']+273.15)**2*p20 + df_h2['P_H2 [bar]']**2*p02 + (df['Temp [degC]']+273.15)*df_h2['P_H2 [bar]']*p11
df_h2['WL [g]'] = df['Wi [g]'] - df_mol['N_H2 [mmol]']*mw_h2/1000 - df_h2['NG_H2O [mmol]']*mw_h2o/1000
df_h2['NL_H2 [mmol]'] = df_h2['S_H2 [mol/kg]']*df_h2['WL [g]']
df_h2.loc[0,'NL_H2 [mmol]'] = 0
df_h2

,P_H2O [bar],NG_H2O [mmol],P_H2 [bar],S_H2 [mol/kg],WL [g],NL_H2 [mmol]
0,15.518495,40.927219,0.000000,0.026419,65.888310,0.000000
1,15.518495,40.927219,43.958176,0.068809,65.656447,4.517752
2,15.518495,40.927219,66.388584,0.090075,65.538134,5.903316
3,15.518495,40.927219,71.379316,0.094773,65.511810,6.208721
4,15.518495,40.927219,77.136046,0.100176,65.481445,6.559701
5,15.518495,40.927219,76.833774,0.099893,65.483040,6.541307
6,7.869734,22.160052,31.357689,0.041635,66.049521,2.749967
7,9.974426,27.466758,46.837497,0.060750,65.872644,4.001751
8,12.504831,33.691290,60.933020,0.079175,65.690218,5.201007
9,15.518495,40.927219,77.136046,0.100176,65.481445,6.559701


In [5]:
# Dataset
data = pd.DataFrame()
data['label'] = ['time']*6 + ['temp']*4 + ['cat']*4
data['T [degC]'] = df['Temp [degC]']
data['P [bar]'] = df_h2['P_H2 [bar]'] + df_h2['P_H2O [bar]']
data['W_cat [g]'] = df['W_cat [g]']
data['Time [h]'] = df['Time [hr]']
data['N_Gly [mmol]'] = df_mol['N_Gly [mmol]']
data['N_KOH [mmol]'] = df_mol['N_KOH [mmol]']
data['N_H2 [mmol]'] = df_mol['N_H2 [mmol]']
data['N_LA [mmol]'] = df_mol['N_LA [mmol]']
data['N_PDO [mmol]'] = df_mol['N_PDO [mmol]']
data['N_FA [mmol]'] = df_mol['N_FA [mmol]']
data['N_GA [mmol]'] = df_mol['N_GA [mmol]']
data['N_ME [mmol]'] = df_mol['N_ME [mmol]']
data['N_EG [mmol]'] = df_mol['N_EG [mmol]']

In [6]:
data.to_csv('data.csv',index=False)

In [7]:
df

,Run [-],Temp [degC],Ni_Gly [mmol],Ni_KOH [mmol],VL_H2O [mL],W_cat [g],Time [hr],Conv [%],LA [%],PDO [%],...,GA [%],ME [%],Others [%],W_H2O [g],Ni_H2O [mmol],EG [%],TOT [%],Wi [g],VL [mL],VG [mL]
0,0,200,250,375,23,0.30,0.0,0.0,0.00000,0.00000,...,0.00000,0.00000,0.00000,23,1277.777778,0.00000,0.000000,66.625,41.253968,103.746032
1,1,200,250,375,23,0.30,0.5,52.8,86.42248,7.17123,...,2.57429,0.55163,0.00000,23,1277.777778,0.55163,97.087385,66.625,41.253968,103.746032
2,2,200,250,375,23,0.30,1.0,82.1,84.89647,6.09013,...,2.19245,0.60901,3.77588,23,1277.777778,0.60901,93.909860,66.625,41.253968,103.746032
3,3,200,250,375,23,0.30,1.5,88.0,85.56818,7.50000,...,2.61364,0.68182,0.79545,23,1277.777778,0.68182,96.477275,66.625,41.253968,103.746032
4,4,200,250,375,23,0.30,2.0,94.4,85.27542,7.20339,...,2.75424,0.84746,0.84746,23,1277.777778,0.84746,96.239405,66.625,41.253968,103.746032
5,5,200,250,375,23,0.30,2.5,98.0,83.16327,7.95918,...,2.44898,0.61224,2.75510,23,1277.777778,0.61224,94.489790,66.625,41.253968,103.746032
6,6,170,250,375,23,0.30,2.0,44.1,83.67347,7.02948,...,1.58730,0.45351,5.20000,23,1277.777778,0.45351,92.970520,66.625,41.253968,103.746032
7,7,180,250,375,23,0.30,2.0,64.7,84.08037,6.49150,...,1.39104,0.46368,5.90000,23,1277.777778,0.46368,92.581145,66.625,41.253968,103.746032
8,8,190,250,375,23,0.30,2.0,81.1,83.10727,7.64488,...,1.97287,0.73983,3.80000,23,1277.777778,0.73983,93.834765,66.625,41.253968,103.746032
9,9,200,250,375,23,0.30,2.0,94.4,85.27542,7.20339,...,2.75424,0.84746,0.84746,23,1277.777778,0.84746,96.239405,66.625,41.253968,103.746032


In [8]:
import numpy as np
N_H2 = np.array([1e-8,1,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220])
vm = (145-41.25)/1e6/N_H2*1e3
P_H2 = np.zeros(len(N_H2))
for i in range(len(N_H2)):
    P_H2[i] = PR(T=200+273.15,V=vm[i],Tc=h2.Tc,Pc=h2.Pc,omega=h2.omega).P
P_H2

vm


NameError: name 'PR' is not defined